# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
valores_ausentes= sinasc.isna().sum()
print(valores_ausentes)


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [3]:
# 3) seu código aqui
colunas_selec = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
df = sinasc[colunas_selec]

val_ausentes_coluna = df.isna().sum()

print(val_ausentes_coluna)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [9]:
# 4) seu código aqui
df = sinasc
df.APGAR5.isna()

0        False
1        False
2        False
3        False
4        False
         ...  
27023    False
27024    False
27025    False
27026    False
27027    False
Name: APGAR5, Length: 27028, dtype: bool

In [12]:
# onde é verdadeiro:

df[df.APGAR5.isna()]


,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
558,1,2515504.0,110002,1,30,1.0,12 anos ou mais,999992.0,0.0,0.0,...,5,562,ATIVO,MUNIC,Buritis,Rondônia,-10.20981,-63.82968,153.0,3265.809
2009,1,NaN,110004,3,35,5.0,1 a 3 anos,999992.0,6.0,2.0,...,3,2028,ATIVO,MUNIC,Cacoal,Rondônia,-11.43387,-61.44294,177.0,3792.998
2340,1,2808560.0,110008,1,29,NaN,8 a 11 anos,999992.0,2.0,0.0,...,5,2363,ATIVO,MUNIC,Costa Marques,Rondônia,-12.43601,-64.23165,145.0,4987.177
2704,1,NaN,110010,3,13,1.0,4 a 7 anos,999992.0,1.0,0.0,...,2,2729,ATIVO,MUNIC,Guajará-Mirim,Rondônia,-10.77388,-65.32395,133.0,24855.724
2705,1,NaN,110010,3,28,1.0,8 a 11 anos,999992.0,4.0,0.0,...,3,2730,ATIVO,MUNIC,Guajará-Mirim,Rondônia,-10.77388,-65.32395,133.0,24855.724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25378,1,NaN,110030,3,20,1.0,12 anos ou mais,999992.0,0.0,0.0,...,1,1492012,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
25864,1,NaN,110030,3,18,5.0,8 a 11 anos,999992.0,0.0,0.0,...,3,1492516,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
26715,1,7218656.0,110149,1,24,5.0,8 a 11 anos,622020.0,3.0,1.0,...,2,1493370,ATIVO,MUNIC,São Francisco do Guaporé,Rondônia,-12.06380,-63.57086,166.0,10959.767
26923,1,2000733.0,120040,1,20,5.0,8 a 11 anos,999992.0,0.0,0.0,...,4,1499180,ATIVO,MUNIC,Porto Velho,Rondônia,-8.76889,-63.83145,87.0,34096.394


In [13]:
df.APGAR5.isna().sum()

103

In [25]:
# vamos remover os dados faltantes da coluna APGAR5

df = df.dropna(subset=['APGAR5'])
df


,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,2604388.0,510340,1,32,2.0,12 anos ou mais,333115.0,1.0,0.0,...,5,2767632,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27024,1,2752654.0,510675,1,19,1.0,8 a 11 anos,999992.0,0.0,0.0,...,5,2772678,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723
27025,1,9659366.0,510787,1,24,2.0,8 a 11 anos,421125.0,0.0,1.0,...,5,2776905,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27026,1,9659366.0,510787,1,21,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2777105,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915


In [27]:
# 5) seu código aqui
df.ESTCIVMAE

0        5.0
1        2.0
2        9.0
3        5.0
4        2.0
        ... 
27023    2.0
27024    1.0
27025    2.0
27026    2.0
27027    5.0
Name: ESTCIVMAE, Length: 26925, dtype: float64

In [36]:
# Vamos substituir os valores ausentes da coluna 'ESTCIVMAE' por 9 que significa ignorado

df['ESTCIVMAE'].fillna('9', inplace=True)
df.to_csv('SINASC_RO_2019.csv', index=False)
df

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,2604388.0,510340,1,32,2.0,12 anos ou mais,333115.0,1.0,0.0,...,5,2767632,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27024,1,2752654.0,510675,1,19,1.0,8 a 11 anos,999992.0,0.0,0.0,...,5,2772678,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723
27025,1,9659366.0,510787,1,24,2.0,8 a 11 anos,421125.0,0.0,1.0,...,5,2776905,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915
27026,1,9659366.0,510787,1,21,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2777105,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915


In [41]:
# verificando se a coluna "ESTCIVMAE" tem dados faltantes depois do tratamento
df['ESTCIVMAE'].isna().any()

False

In [53]:
# vamos fazer a mesma coisa para a coluna "CONSULTAS"
df.CONSULTAS.isna().sum() #analisamos que a coluna "CONSULTAS" não possui dados faltantes para fazer o tratamento.



0

In [57]:
# 6) Substituindo os valores faltantes por '0':

df['QTDFILVIVO'].fillna('0', inplace=True)
df.to_csv('SINASC_RO_2019.csv', index=False)
df.QTDFILVIVO.isna().sum() #analisamos que a coluna "QTDFILVIVO" não possui mais dados faltantes 

0

In [61]:
# 7) seu código aqui
coluna = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
df[coluna].isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

Analisando o resultado de dados faltantes observamos que "ESCMAE" não é um dado importante, pois não influencia em nada na saúde do bebe, além de significar 1% faltante do total de dados. a coluna "GESTACAO" E "GRAVIDEZ" apesar de serem importantes tambem representam uma baixa quantidade de dados faltantes capaz de impactar a análise, então é prefirível preencher como ignorado

In [62]:
df['GESTACAO'].fillna('9', inplace=True)
df.to_csv('SINASC_RO_2019.csv', index=False)

In [63]:
df['GRAVIDEZ'].fillna('9', inplace=True)
df.to_csv('SINASC_RO_2019.csv', index=False)

In [65]:
df['ESCMAE'].fillna('9', inplace=True)
df.to_csv('SINASC_RO_2019.csv', index=False)

Vamos verificar se os dados foram tratados corretamente:

In [66]:
df[coluna].isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

Vamos Categorizar alguns dados e criar uma coluna chamada "Apgar_categoria":

In [99]:
# 8)
import numpy as np
df.loc[(df['APGAR5'] > 7) & (df['APGAR5'] <= 10) , 'Apgar_categoria'] = 'normal'
df.loc[(df['APGAR5'] >= 6) & (df['APGAR5'] <= 7) , 'Apgar_categoria'] = 'asfixia leve'
df.loc[(df['APGAR5'] >= 4) & (df['APGAR5'] <= 5) , 'Apgar_categoria'] = 'asfixia moderada'
df.loc[(df['APGAR5'] >= 0) & (df['APGAR5'] <= 3) , 'Apgar_categoria'] = 'asfixia severa'
df[['APGAR5', 'Apgar_categoria']]

,APGAR5,Apgar_categoria
0,10.0,normal
1,9.0,normal
2,10.0,normal
3,10.0,normal
4,10.0,normal
...,...,...
27023,9.0,normal
27024,9.0,normal
27025,10.0,normal
27026,9.0,normal


Vamos calcular a frequencia dessa categorização:

In [107]:
normal = df['Apgar_categoria'].value_counts()['normal']
leve = df['Apgar_categoria'].value_counts()['asfixia leve']
moderada = df['Apgar_categoria'].value_counts()['asfixia moderada']
severa = df['Apgar_categoria'].value_counts()['asfixia severa']
print(f' {normal} crianças nasceram sem asfixia e classificada como normal.' )
print(f' {leve} crianças nasceram com asfixia leve.' )
print(f' {moderada} crianças nasceram com asfixia moderada.' )
print(f' {severa} crianças nasceram com asfixia severa.' )


 26463 crianças nasceram sem asfixia e classificada como normal.
 320 crianças nasceram com asfixia leve.
 68 crianças nasceram com asfixia moderada.
 74 crianças nasceram com asfixia severa.


Vamos renomear as colunas, transformando todas em letras minúsculas.

In [114]:
df.rename(columns=lambda x: x.lower(), inplace = True)
df

,origem,codestab,codmunnasc,locnasc,idademae,estcivmae,escmae,codocupmae,qtdfilvivo,qtdfilmort,...,contador,munresstatus,munrestipo,munresnome,munresuf,munreslat,munreslon,munresalt,munresarea,apgarcategoria
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273,normal
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,2604388.0,510340,1,32,2.0,12 anos ou mais,333115.0,1.0,0.0,...,2767632,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,normal
27024,1,2752654.0,510675,1,19,1.0,8 a 11 anos,999992.0,0.0,0.0,...,2772678,ATIVO,MUNIC,Chupinguaia,Rondônia,-12.55684,-60.90185,340.0,5126.723,normal
27025,1,9659366.0,510787,1,24,2.0,8 a 11 anos,421125.0,0.0,1.0,...,2776905,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,normal
27026,1,9659366.0,510787,1,21,2.0,8 a 11 anos,999992.0,1.0,0.0,...,2777105,ATIVO,MUNIC,Vilhena,Rondônia,-12.74137,-60.13858,595.0,11518.915,normal
